In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
expenditures = ScrapedExpense.objects.all()

descriptions = expenditures.values('description').annotate(sum=Sum('amount')).order_by('-sum')

segments = {
    'television': {
        'terms': ['tv', 'television', 'cable'],
        'false_terms': ['gotv', 'allocable'],
        'matches': [],
        'total': 0.
    },
    'radio': {
        'terms': ['radio'],
        'false_terms': [],
        'matches': [],
        'total': 0.
    },
    'digital': {
        'terms': ['digital', 'online', 'social', 'facebook', 'google'],
        'false_terms': [],
        'matches': [],
        'total': 0.
    }
}

unmatched_descriptions = []

for description in descriptions:
    found = False
    for segment in segments:
        for term in segments[segment]['terms']:
            if description['description'].lower().find(term) != -1:
                false_found = False
                for false_term in segments[segment]['false_terms']:
                    if description['description'].lower().find(false_term) != -1:
                        false_found = True
                if not false_found:
                    segments[segment]['matches'].append(description['description'])
                    segments[segment]['total'] += float(description['sum'])
                found = True
    if not found:
        unmatched_descriptions.append(description['description'])

for segment in segments:
    print 'Found %d terms matching %s for %s' % (
        len(segments[segment]['matches']), segment, locale.currency(segments[segment]['total'], grouping=True))

print 'Unmatched descriptions:\n%s' % unmatched_descriptions

Found 279 terms matching television for $17,356,490.95
Found 153 terms matching radio for $1,656,542.25
Found 591 terms matching digital for $7,570,304.97
Unmatched descriptions:
[u'CONTRIBUTION', u'DONATION', u'SIGNATURE GATHERING', u'CONSULTING', u'WAGES', u'MEDIA ADVERTISING', u'MEDIA BUY', u'CAMPAIGN CONTRIBUTION', u'OTHER: LEGAL SERVICES', u'LOAN REPAYMENT', u'BROADCAST ADVERTISING', u'POSTAGE', u'PAYROLL TAXES', u'OTHER: FUNDRAISING CONSULTING', u'SIGNATURE GATHERING FEE', u'CONTRIBUTION.', u'POLITICAL CONTRIBUTION', u'CONTRIBUTIONS: UNLIMITED TRANSFER', u'OTHER: FUNDRAISING - CONSULTING', u'ADVERTISING', u'JAY INSLEE FOR WASHINGTON/MAIL PLAN', u'PRINTING', u'ADMINISTRATIVE: SALARY', u'SALARY', u'BROADCAST ADVERTISING - SEATTLE', u'TRANSFER TO SURPLUS ACCOUNT', u'MEDIA BUY (SEE MEMO)', u'', u'FUNDRAISING: DIRECT MAIL PRODUCTION', u'OTHER: TRAVEL', u'SURPLUS TRANSFER', u'POLLING', u'PAID SIGNATURES', u'OTHER: TECHNOLOGY SERVICES', u'CONTRIBUTION TO COMMITTEE', u'CONTRIBUTION TO CA